In [ ]:
!pip install unsloth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 6.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.3/359.3 kB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.0/288.0 kB 30.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/899.7 MB 241.2 MB/s eta 0:00:04

In [ ]:
from unsloth import FastLanguageModel

model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name,
    load_in_4bit=True,
    max_seq_length=256,
    dtype=None,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 8, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj",
                    "gate_proj", "up_proj", "down_proj"], #Mamba out_proj and conv1d layers should not be included here see https://github.com/huggingface/peft/pull/2562
    lora_alpha = 8,
    lora_dropout = 0.1,
    use_gradient_checkpointing = False,
    random_state = 1407,
)

In [ ]:
alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }

from datasets import load_dataset
dataset = load_dataset("yahma/alpaca-cleaned", split = "train")
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [ ]:
from trl import SFTConfig, SFTTrainer
trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=256,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=SFTConfig(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=8,
        warmup_steps=5,
        # num_train_epochs = 1, # Set this for 1 full training run.
        max_steps=10,
        learning_rate=2e-4,
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.001,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        report_to="none",  # Use TrackIO/WandB etc
    ),
)

In [ ]:
trainer_stats = trainer.train()

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
tokenizer.batch_decode(outputs)

In [ ]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "Continue the fibonnaci sequence.", # instruction
        "1, 1, 2, 3, 5, 8", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)

In [ ]:
model.save_pretrained("lora_model")  # Local saving
tokenizer.save_pretrained("lora_model")

In [ ]:
device_map="auto"

model.save_pretrained_merged("merged_model", tokenizer, save_method = "merged_16bit",)

In [ ]:
# STEP 1: Clone llama.cpp repository

!git clone https://github.com/ggerganov/llama.cpp.git

# STEP 2: Build llama.cpp (for quantization binary)

%cd llama.cpp
!cmake -B build
!cmake --build build --config Release -j
%cd ..

In [ ]:
%%bash

# For BF16:
!python llama.cpp/convert_hf_to_gguf.py merged_model \
    --outfile model-BF16.gguf --outtype bf16 \
    --split-max-size 50G

# For Q8_0:
!python llama.cpp/convert_hf_to_gguf.py merged_model \
    --outfile gemma-2-2b-Q8_0.gguf --outtype q4_0 \
    --split-max-size 50G

In [ ]:
%%bash

# 🚀 Build & quantize BF16 → Q4_K_M (auto-detect quantizer)
cd /kaggle/working/llama.cpp && \
mkdir -p build && cd build && \
cmake .. && \
cmake --build . -j && \
cd bin && \
QUANT_BIN=$(ls -1 *quantize* 2>/dev/null | head -n 1) && \
echo "🧠 Using quantizer: $QUANT_BIN" && \
./"$QUANT_BIN" /kaggle/working/model-BF16.gguf \
               /kaggle/working/Phi-3-mini-4k-instruct-q4_K_M.gguf \
               q4_K_M && \
ls -lh /kaggle/working/Phi-3-mini-4k-instruct-q4_K_M.gguf


In [ ]:
%%bash

cd /kaggle/working/llama.cpp/build/bin
./llama-quantize /kaggle/working/Phi-3-mini-4k-instruct-BF16.gguf \
                 /kaggle/working/Phi-3-mini-4k-instruct-q3_K.gguf \
                 q3_K

In [ ]:
!hf upload ss-lab/Phi-3-mini-4k-instruct-GGUF \
    /kaggle/working/Phi-3-mini-4k-instruct-q3_K.gguf

In [ ]:
!rm /kaggle/working/Phi-3-mini-4k-instruct-q3_K.gguf